March Madness ML Project

In [37]:
# lib imports copied from lab 4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Load the LinearRegression class from scikit-learn's linear_model module
# Also load the PolynomialFeatures transformer and OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [38]:
RSDR = "mens-march-mania-2022\MDataFiles_Stage1\MRegularSeasonDetailedResults.csv"
Teams = "mens-march-mania-2022\MDataFiles_Stage1\MTeams.csv"
# For the fisrt iteration lets omit the seeds
Seeds = "mens-march-mania-2022\MDataFiles_Stage1\MNCAATourneySeeds.csv"
SeedRoundSlots = "mens-march-mania-2022\MDataFiles_Stage1\MNCAATourneySeedRoundSlots.csv"

ConferenceTourney = "mens-march-mania-2022\MDataFiles_Stage1\MConferenceTourneyGames.csv"
MasseyOrd = "mens-march-mania-2022\MDataFiles_Stage1\MMasseyOrdinals.csv"
Coaches = "mens-march-mania-2022\MDataFiles_Stage1\MTeamCoaches.csv"
TeamConferences = "mens-march-mania-2022\MDataFiles_Stage1\MTeamConferences.csv"


# All databases that could potentially have usable data
RSDR = pd.read_csv(RSDR, na_values = ["?"]).dropna()
Teams = pd.read_csv(Teams, na_values = ["?"]).dropna()
Seeds = pd.read_csv(Seeds, na_values = ["?"]).dropna()
SeedRoundSlots = pd.read_csv(SeedRoundSlots, na_values = ["?"]).dropna()

ConferenceTourney = pd.read_csv(ConferenceTourney, na_values = ["?"]).dropna()
MasseyOrd = pd.read_csv(MasseyOrd, na_values = ["?"]).dropna()
Coaches = pd.read_csv(Coaches, na_values = ["?"]).dropna()


# RSDR.info()
RSDR.head()
# Teams.info()
# Teams.head()
# ConferenceTourney.info()
# ConferenceTourney.head()
# MasseyOrd.info()
# MasseyOrd.head(20)
# Coaches.info()
# Coaches.head(20)

# Due to player turn over we want to limit the data to "more recent" data of 2015 and onward and reset the index
RSDR = RSDR[(RSDR['Season'] > 2015)].reset_index()
RSDR.drop(['index'], axis= 1, inplace= True)
MasseyOrd = MasseyOrd[(MasseyOrd['Season'] > 2015)].reset_index()
MasseyOrd.drop(['index'], axis= 1, inplace= True)
Seeds = Seeds[(Seeds['Season'] > 2015)].reset_index()
Seeds.drop(['index'], axis= 1, inplace= True)

# Need to replace the location objects with random number values
RSDR['WLoc'].replace({'H' : 0, 'A' : 1, 'N' : 2}, inplace = True)

#Checking to see if we have any null values in the current semi-cleaned data set
# sns.heatmap(RSDR.isnull(), yticklabels= False, cbar= False, cmap= "viridis")
# sns.heatmap(MasseyOrd.isnull(), yticklabels= False, cbar= False, cmap= "viridis")
# sns.heatmap(Seeds.isnull(), yticklabels= False, cbar= False, cmap= "viridis")


